In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
# initial parameters
epochs = 100
lr = 1e-3
batch_size = 64
img_dims = (96,96,3)

In [4]:
data = []
labels = []

In [5]:
# load image files from the dataset
image_files = [f for f in glob.glob(r"D:\Computer courses\AI Projects\PRAICP-1001-GenderDetc\Data\images" + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)

In [6]:
# converting images to arrays and labelling the categories
for img in image_files:

    image = cv2.imread(img)
    
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2] 
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label]) 

In [7]:
# pre-processing
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [8]:
# split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)

trainY = to_categorical(trainY, num_classes=2) 
testY = to_categorical(testY, num_classes=2)

In [9]:
# augmenting datset 
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [10]:
# define model
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
        inputShape = (depth, height, width)
        chanDim = 1
    
    # The axis that should be normalized, after a Conv2D layer with data_format="channels_first", 
    # set axis=1 in BatchNormalization.

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model

In [11]:
# build model
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

In [17]:
model.compile(loss='binary_crossentropy', 
              optimizer = (tf.keras.optimizers.Adam(learning_rate=0.0001)), 
#               optimizer = 'adam',
              metrics = ['accuracy'])

In [18]:
checkpoint = ModelCheckpoint(filepath = 'gender_detection.model', verbose = 1, save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss',  
                           patience=10,         
                           restore_best_weights=True)

In [20]:
# train the model
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=2)

Epoch 1/100
28/28 - 23s - loss: 0.7562 - accuracy: 0.6917 - val_loss: 0.6878 - val_accuracy: 0.5455 - 23s/epoch - 825ms/step
Epoch 2/100
28/28 - 22s - loss: 0.6506 - accuracy: 0.7479 - val_loss: 0.6863 - val_accuracy: 0.4978 - 22s/epoch - 798ms/step
Epoch 3/100
28/28 - 22s - loss: 0.5994 - accuracy: 0.7732 - val_loss: 0.6960 - val_accuracy: 0.5476 - 22s/epoch - 797ms/step
Epoch 4/100
28/28 - 22s - loss: 0.5265 - accuracy: 0.8052 - val_loss: 0.6622 - val_accuracy: 0.6667 - 22s/epoch - 798ms/step
Epoch 5/100
28/28 - 22s - loss: 0.4969 - accuracy: 0.8119 - val_loss: 0.6565 - val_accuracy: 0.5887 - 22s/epoch - 795ms/step
Epoch 6/100
28/28 - 22s - loss: 0.4990 - accuracy: 0.8125 - val_loss: 0.6080 - val_accuracy: 0.7446 - 22s/epoch - 799ms/step
Epoch 7/100
28/28 - 23s - loss: 0.4678 - accuracy: 0.8355 - val_loss: 0.7569 - val_accuracy: 0.5087 - 23s/epoch - 831ms/step
Epoch 8/100
28/28 - 22s - loss: 0.4467 - accuracy: 0.8349 - val_loss: 0.8081 - val_accuracy: 0.5130 - 22s/epoch - 792ms/step


28/28 - 22s - loss: 0.1567 - accuracy: 0.9444 - val_loss: 0.3147 - val_accuracy: 0.9091 - 22s/epoch - 785ms/step
Epoch 67/100
28/28 - 22s - loss: 0.1668 - accuracy: 0.9382 - val_loss: 0.4034 - val_accuracy: 0.8788 - 22s/epoch - 786ms/step
Epoch 68/100
28/28 - 22s - loss: 0.1408 - accuracy: 0.9422 - val_loss: 0.2789 - val_accuracy: 0.9113 - 22s/epoch - 789ms/step
Epoch 69/100
28/28 - 22s - loss: 0.1381 - accuracy: 0.9478 - val_loss: 0.1635 - val_accuracy: 0.9416 - 22s/epoch - 786ms/step
Epoch 70/100
28/28 - 22s - loss: 0.1489 - accuracy: 0.9500 - val_loss: 0.2270 - val_accuracy: 0.9264 - 22s/epoch - 784ms/step
Epoch 71/100
28/28 - 22s - loss: 0.1562 - accuracy: 0.9467 - val_loss: 0.1099 - val_accuracy: 0.9719 - 22s/epoch - 784ms/step
Epoch 72/100
28/28 - 22s - loss: 0.1252 - accuracy: 0.9596 - val_loss: 0.2507 - val_accuracy: 0.9242 - 22s/epoch - 784ms/step
Epoch 73/100
28/28 - 22s - loss: 0.1510 - accuracy: 0.9478 - val_loss: 0.1583 - val_accuracy: 0.9459 - 22s/epoch - 797ms/step
Epoch

In [21]:
# save the model to disk
model.save('gender_detection.model')

In [22]:
tscore = model.evaluate(trainX, trainY, verbose=0)
taccuracy = 100 * tscore[1]
tloss = 100 * tscore[0]
print(tscore)
print(tloss)
print(taccuracy)

[0.02435472048819065, 0.9918699264526367]
2.435472048819065
99.18699264526367


In [23]:
vscore = model.evaluate(testX,testY, verbose=0)
vaccuracy = 100 * vscore[1]
vloss = 100 * vscore[0]
print(vscore)
print(vloss)
print(vaccuracy)

[0.09241681545972824, 0.9718614816665649]
9.241681545972824
97.1861481666565


In [5]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os
import cvlib as cv
                    
# load model
model = load_model('gender_detection.model')

# open webcam
webcam = cv2.VideoCapture(0)
    
classes = ['man','woman']

# loop through frames
while webcam.isOpened():

    # read frame from webcam 
    status, frame = webcam.read()

    # apply face detection
    face, confidence = cv.detect_face(frame)


    # loop through detected faces
    for idx, f in enumerate(face):

        # get corner points of face rectangle        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw rectangle over face
        cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)

        # crop the detected face region
        face_crop = np.copy(frame[startY:endY,startX:endX])

        if (face_crop.shape[0]) < 10 or (face_crop.shape[1]) < 10:
            continue

        # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        conf = model.predict(face_crop)[0] # model.predict return a 2D matrix, ex: [[9.9993384e-01 7.4850512e-05]]

        # get label with max accuracy
        idx = np.argmax(conf)
        label = classes[idx]

        label = "{}: {:.2f}%".format(label, conf[idx] * 100)

        Y = startY - 10 if startY - 10 > 10 else startY + 10

        # write label and confidence above face rectangle
        cv2.putText(frame, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 255, 0), 2)

    # display output
    cv2.imshow("gender detection", frame)

    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release resources
webcam.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 59ms/step
